## 使用LCEL来自定义路由链

In [7]:
# 导入必要的库
from langchain_core.output_parsers import StrOutputParser  # 导入字符串输出解析器
from langchain_core.prompts import PromptTemplate  # 导入提示模板
import os  # 导入os库
from langchain_deepseek import ChatDeepSeek
# 导入RunnableLambda用于创建可运行的函数链
from langchain_core.runnables import RunnableLambda


claudeLLM = ChatDeepSeek(
    model="Pro/deepseek-ai/DeepSeek-V3",
    temperature=0,
    api_key=os.environ.get("DEEPSEEK_API_KEY"),
    api_base=os.environ.get("DEEPSEEK_API_BASE"),
)

# 创建分类链 - 用于确定问题类型
chain = (
    # 创建提示模板，要求模型将问题分类为LangChain、Anthropic或Other
    PromptTemplate.from_template(
        """根据下面的用户问题，将其分类为 `LangChain`、`Anthropic` 或 `Other`。

请只回复一个词作为答案。

<question>
{question}
</question>

分类结果:"""
    )
    | claudeLLM  # 将提示发送给Claude模型
    | StrOutputParser()  # 解析模型的输出为纯文本
)

# 创建LangChain专家链 - 模拟Harrison Chase(LangChain创始人)的回答风格
langchain_chain = PromptTemplate.from_template(
    """你将扮演一位LangChain专家。请以他的视角回答问题。 \
你的回答必须以"正如Harrison Chase告诉我的"开头，否则你会受到惩罚。 \
请回答以下问题:

问题: {question}
回答:"""
) | claudeLLM  # 将提示发送给



# 创建Anthropic专家链 - 模拟Dario Amodei(Anthropic创始人)的回答风格
anthropic_chain = PromptTemplate.from_template(
    """你将扮演一位一位Anthropic专家。请以他的视角回答问题。 \
你的回答必须以"正如Dario Amodei告诉我的"开头，否则你会受到惩罚。 \
请回答以下问题:

问题: {question}
回答:"""
) | claudeLLM  

# 创建通用回答链 - 用于处理其他类型的问题
general_chain = PromptTemplate.from_template(
    """请回答以下问题:

问题: {question}
回答:"""
) | claudeLLM 

# 自定义路由函数 - 根据问题分类结果选择合适的回答链
def route(info):
    print(info)   # 打印分类结果
    # 根据分类结果选择相应的专家链
    if "anthropic" in info["topic"].lower():  # 如果问题与Anthropic相关
        print("claude")
        return anthropic_chain  # 使用Anthropic专家链
    elif "langchain" in info["topic"].lower():  # 如果问题与LangChain相关
        print("langchain")
        return langchain_chain  # 使用LangChain专家链
    else:  # 其他类型的问题
        print("general")
        return general_chain # 使用通用回答链



# 创建完整的处理链
# 1. 首先将问题分类并保留原始问题
# 2. 然后根据分类结果路由到相应的专家链处理
full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

# 调用完整链处理用户问题
# 这个问题会被分类为Anthropic相关，然后由anthropic_chain处理
full_chain.invoke({"question": "我该如何使用langchain?"})

{'topic': 'LangChain', 'question': '我该如何使用langchain?'}
langchain


AIMessage(content='正如Harrison Chase告诉我的，使用LangChain的第一步是理解其核心概念和组件。LangChain是一个用于构建基于语言模型的应用的框架，它允许你将语言模型与其他数据源和工具集成起来。以下是一些基本步骤来开始使用LangChain：\n\n1. **安装LangChain**：首先，你需要通过pip安装LangChain库。你可以使用以下命令进行安装：\n   ```bash\n   pip install langchain\n   ```\n\n2. **导入必要的模块**：在你的Python脚本中，导入LangChain的核心模块。例如：\n   ```python\n   from langchain import LLMChain, PromptTemplate\n   ```\n\n3. **创建提示模板**：提示模板是LangChain中的一个重要概念，它定义了如何将用户输入转换为语言模型的输入。你可以使用`PromptTemplate`类来创建提示模板：\n   ```python\n   prompt = PromptTemplate(\n       input_variables=["topic"],\n       template="请写一篇关于{topic}的文章。"\n   )\n   ```\n\n4. **初始化语言模型**：LangChain支持多种语言模型，包括OpenAI的GPT-3、GPT-4等。你需要初始化一个语言模型实例：\n   ```python\n   from langchain.llms import OpenAI\n   llm = OpenAI(api_key="your_openai_api_key")\n   ```\n\n5. **创建链**：链是LangChain中的另一个核心概念，它将提示模板和语言模型连接起来。你可以使用`LLMChain`类来创建链：\n   ```python\n   chain = LLMChain(llm=llm, prompt=prompt)\n   ```\n\n6. **运行链**：最后，你可以通过调用链的`run`方法来生成输出：\n   ```python\n   output = chain.run(top